In [ ]:
import os
import gc
import random
import pickle
import math
import time
import itertools

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras.models import load_model

%load_ext autoreload
%autoreload 2
import lib.model as model
import lib.data_processing as dp
import lib.simulator as simulator

In [ ]:
# 데이터 로드
ROOT_DIR = "../"
MODEL_DIR = ROOT_DIR + "Model/"
OUTPUT_DIR = ROOT_DIR + "Output/"

DATA_DIR = ROOT_DIR + "Data/"
TRAIN_DIR = DATA_DIR + "spot/usdt/train_20.12.31/"
TEST_DIR = DATA_DIR + "spot/usdt/test_22.05.31/"

train_datas, test_datas = dp.load_data(TRAIN_DIR, TEST_DIR)

In [ ]:
# 모델 정의 및 생성
X_len = 128
y_len = 1

time_windows = [1,2,4,8,16]
y_windows = [64]

X_shape = (X_len, len(time_windows))
y_shape = (len(y_windows),)

print(X_shape, y_shape)
print(time_windows, y_windows)

lr = 1e-3
data_shapes = [X_shape, y_shape]
optimizer = Adam(learning_rate=lr)
loss = 'mse'
metrics = [model.sign_acc]
model_configs = [optimizer, loss, metrics]
predictor = model.inception(data_shapes, model_configs)
#predictor = model.vgg(data_shapes, model_configs)
#predictor = model.lstm(data_shapes, model_configs)

# 모델 모양 출력
plot_model(predictor, to_file=OUTPUT_DIR + 'model/structure.png', show_shapes=True, expand_nested=True)

In [ ]:
# 데이터 로더 정의
batch_size = 1024
datas = (train_datas, test_datas)
data_infos = (batch_size, X_len, y_len, time_windows, y_windows)

dataloader = dp.DataLoader(datas, data_shapes, data_infos)

# history 초기화
history = {}
history['step'] = 0
history['train_losses'] = []
history['test_losses'] = []
history['train_accs'] = []
history['test_accs'] = []

In [ ]:
# 테스트 데이터 시각화
plt.clf()
plt.title("X_test[0]")
plt.plot(dataloader.X_test[0,:,:])
plt.show()

plt.clf()
plt.title("y_test[:128]")
plt.plot(dataloader.y_test[0:X_len])
plt.show()

In [ ]:
# 모델 로드
#predictor = load_model(MODEL_DIR + 'X128_5_limit0.43.h5', compile=False)
predictor = load_model(OUTPUT_DIR + 'model/predictor.h5', compile=False)
predictor.compile(optimizer=optimizer, loss=loss, metrics=metrics)

try:
    with open(OUTPUT_DIR + 'history.pkl', 'rb') as f:
        history = pickle.load(f)
except:
    print("NO history")

In [ ]:
# 전이 학습용 가중치 동결
for i_layer, layer in enumerate(predictor.layers):
    if "Dense" in str(layer):
        break
    else:
        predictor.layers[i_layer].trainable = False
        print(layer, predictor.layers[i_layer].trainable)

In [ ]:
# 학습 진행
X_train = dataloader.X_train; y_train = dataloader.y_train
X_test = dataloader.X_test; y_test = dataloader.y_test

eval_time = time.time()
result_time = time.time()
while(True):
    X_data, y_data = dataloader.get_train_data()
    predictor.train_on_batch(X_data, y_data)

    if history['step'] > 1000 and time.time() - eval_time > 60:
        eval_time = time.time()
        train_eval = predictor.evaluate(X_train, y_train, verbose=0)
        test_eval = predictor.evaluate(X_test, y_test, verbose=0)
        history['train_losses'].append(train_eval[0])
        history['test_losses'].append(test_eval[0])
        history['train_accs'].append(train_eval[1])
        history['test_accs'].append(test_eval[1])

        print('step: ' + str(history['step']))
        print("train_loss: %f, train_acc: %f" % (history['train_losses'][-1], history['train_accs'][-1]))
        print("test_loss:  %f, test_acc:  %f" % (history['test_losses'][-1], history['test_accs'][-1]))

        if np.max(history['test_accs']) == history['test_accs'][-1]:
            predictor.save(OUTPUT_DIR + 'model/best_predictor.h5')

    if history['step'] == 0 or time.time() - result_time > 60:
        lr = lr * (1-0.01)
        K.set_value(predictor.optimizer.learning_rate, lr)
        fig_size = 1024
        result_time = time.time()
        y_pred = predictor.predict(X_train[:fig_size], verbose=0)
        plt.clf()
        plt.title("y_train and y_pred")
        plt.xlabel("time (minute)")
        plt.ylabel("predict")
        plt.plot(list(range(fig_size)), y_train[:fig_size])
        plt.plot(list(range(fig_size)), y_pred[:fig_size])
        plt.draw()
        plt.gcf().savefig(OUTPUT_DIR + 'train' + str(history['step']) + '.png')

        y_pred = predictor.predict(X_test[:fig_size], verbose=0)
        plt.clf()
        plt.title("y_test and y_pred")
        plt.xlabel("time (minute)")
        plt.ylabel("predict")
        plt.plot(list(range(fig_size)), y_test[:fig_size])
        plt.plot(list(range(fig_size)), y_pred[:fig_size])
        plt.draw()
        plt.gcf().savefig(OUTPUT_DIR + 'test' + str(history['step']) + '.png')
        
        plt.clf()
        plt.title("losses")
        plt.xlabel("train time (minute)")
        plt.ylabel("mse")
        plt.plot(list(range(len(history['train_losses']))), history['train_losses'])
        plt.plot(list(range(len(history['test_losses']))), history['test_losses'])
        plt.draw()
        plt.gcf().savefig(OUTPUT_DIR + 'losses.png')
        
        plt.clf()
        plt.title("sign accs")
        plt.xlabel("train time (minute)")
        plt.ylabel("acc")
        plt.plot(list(range(len(history['train_accs']))), history['train_accs'])
        plt.plot(list(range(len(history['test_accs']))), history['test_accs'])
        plt.draw()
        plt.gcf().savefig(OUTPUT_DIR + 'accs.png')
        
        with open(OUTPUT_DIR + 'history.pkl', 'wb') as f:
            pickle.dump(history, f)
        
    if history['step'] == 0 or history['step'] % 1000 == 0:
        predictor.save(OUTPUT_DIR + 'model/predictor.h5')
    
    history['step'] += 1

In [ ]:
# 마지막 모델 상태 저장
predictor.save(OUTPUT_DIR + 'model/predictor.h5')

In [ ]:
# 예측 및 기본 수익률 계산
symbol = "DOGE"
ohlcv_data = test_datas[symbol]
y_data, y_pred = simulator.get_real_pred(predictor, dataloader, symbol)

# 예측 데이터 시각화
plt.clf()
plt.plot(list(range(len(y_data))), y_data)
plt.plot(list(range(len(y_pred))), y_pred)
plt.show()

# Trader 시뮬레이팅
fee_rate = 0.0
margin_rate = 1.0
threshold = [0.0, 0.0]
trading_conditions = [fee_rate, margin_rate, threshold]
trader = simulator.TraderFutureMarket(ohlcv_data, y_pred, trading_conditions)
'''fee_rate = 0.02 / 100
limit_rate = 0.42 / 100
trading_conditions = [fee_rate, margin_rate, threshold, limit_rate]
trader = simulator.TraderFutureLimit(ohlcv_data, y_pred, trading_conditions)
'''
trading_info, money_list, value_list = trader.get_yield()

# trade_log, trade_num, win_rate, trading_yield
print("거래횟수: %s, %s 회" % (trading_info[1][0], trading_info[1][1]))
print("승률: %s, %s %%" % (trading_info[2][0], trading_info[2][1]))
trading_yield = (money_list[-1]/money_list[0] - 1) * 100
print("수익률: %s %%" % trading_yield)

# 결과 시각화
plt.clf()
plt.title("value and asset")
plt.xlabel("time(minute)")
plt.ylabel("scale")
plt.plot(list(range(len(value_list))), value_list/value_list[0])
plt.plot(list(range(len(money_list))), money_list/money_list[0])

trade_log = np.array(trading_info[0]) * (np.max(value_list)/value_list[0] - np.min(value_list)/value_list[0]) + np.min(value_list)/value_list[0]
plt.show()
print("Last y_pred: ", y_pred[-1])

In [ ]:
# threshold 최적화
yields = []
win_rates = []

fee_rate = 0.04 / 100
margin_rate = 1.0
limit_rate = 0.0
mul_threshold = 1.0 / 100
for i in range(100):
    threshold = [0.0 + mul_threshold * i, 0.0 - mul_threshold * i]
    trading_conditions = [fee_rate, margin_rate, threshold]
    trader = simulator.TraderFutureMarket(ohlcv_data, y_pred, trading_conditions)
    trading_info, money_list, value_list = trader.get_yield()
    win_rates.append(trading_info[2])
    yields.append((money_list[-1]/money_list[0] - 1) * 100)

    plt.clf()
    plt.plot(list(range(len(value_list))), value_list/value_list[0])
    plt.plot(list(range(len(money_list))), money_list/money_list[0])
    plt.show()

In [ ]:
# threshold 최대 그래프 출력
max_idx = np.argmax(yields) 
max_yield = yields[max_idx]

threshold = [0.0 + mul_threshold * max_idx, 0.0 - mul_threshold * max_idx]
trading_conditions = [fee_rate, margin_rate, threshold]
trader = simulator.TraderFutureMarket(ohlcv_data, y_pred, trading_conditions)
trading_info, money_list, value_list = trader.get_yield()

print("거래횟수: %s, %s 회" % (trading_info[1][0], trading_info[1][1]))
print("승률: %s, %s %%" % (trading_info[2][0], trading_info[2][1]))
trading_yield = (money_list[-1]/money_list[0] - 1) * 100
print("수익률: %s %%" % trading_yield)

plt.clf()
plt.plot(list(range(len(value_list))), value_list/value_list[0])
plt.plot(list(range(len(money_list))), money_list/money_list[0])
plt.show()

print("max_idx, max_yield: ", max_idx, max_yield)
print("threshold: ", threshold)
print("limit_rate: ", limit_rate * 100)

plt.clf()
plt.plot(list(range(len(yields))), yields)
plt.show()

plt.clf()
plt.plot(list(range(len(win_rates))), win_rates)
plt.show()

In [ ]:
# limit_rate 최적화
yields = []
win_rates = []

fee_rate = 0.02 / 100
margin_rate = 1.0
threshold = [0.0, 0.0]
mul_limit_rate = 1.0 / 100
for i in range(100):
    limit_rate = mul_limit_rate * i / 100
    trading_conditions = [fee_rate, margin_rate, threshold, limit_rate]
    trader = simulator.TraderFutureLimit(ohlcv_data, y_pred, trading_conditions)
    trading_info, money_list, value_list = trader.get_yield()
    win_rates.append(trading_info[2])
    yields.append((money_list[-1]/money_list[0] - 1) * 100)

    plt.clf()
    plt.plot(list(range(len(value_list))), value_list/value_list[0])
    plt.plot(list(range(len(money_list))), money_list/money_list[0])
    plt.show()

In [ ]:
# limit rate 최대 그래프 출력
max_idx = np.argmax(yields[1:]) + 1
max_yield = yields[max_idx]

threshold = [0.0, 0.0]
limit_rate = mul_limit_rate * max_idx / 100

trading_conditions = [fee_rate, margin_rate, threshold, limit_rate]
trader = simulator.TraderFutureLimit(ohlcv_data, y_pred, trading_conditions)
trading_info, money_list, value_list = trader.get_yield()

print("거래횟수: %s, %s 회" % (trading_info[1][0], trading_info[1][1]))
print("승률: %s, %s %%" % (trading_info[2][0], trading_info[2][1]))
trading_yield = (money_list[-1]/money_list[0] - 1) * 100
print("수익률: %s %%" % trading_yield)

plt.clf()
plt.plot(list(range(len(value_list))), value_list/value_list[0])
plt.plot(list(range(len(money_list))), money_list/money_list[0])
plt.show()

print("max_idx, max_yield: ", max_idx, max_yield)
print("threshold: ", threshold)
print("limit_rate: ", limit_rate * 100)

plt.clf()
plt.plot(list(range(len(yields[1:]))), yields[1:])
plt.show()

plt.clf()
plt.plot(list(range(len(win_rates[1:]))), win_rates[1:])
plt.show()

In [ ]:
# 현물
class StockTrader:
    def __init__(self, predictor, X_data, y_data, ohlcv_data):
        self.today = 0
        self.money = 10000000 # 예수금
        self.amount = 0 # 보유량
        self.value_today = 0 # 오늘가
        self.value_tomorrow = 0 # 내일가
        self.rate_tomorrow = 0
        self.money_total = self.money

        self.fee_rate = 0.00
        self.fee_rate = fee_rate
        
        self.predictor = predictor # 예측모델
        self.X_data = X_data
        self.y_data = y_data
        self.ohlcv_data = ohlcv_data
        self.y_pred = self.y_data
        #self.y_pred = self.predictor.predict(self.X_data)
        self.y_pred = y_pred_global # 전역 변수
        # 0~99 까지의 rate를 통해(즉, 100의 데이터는 알고 있다.) 101의 데이터를 얻어낸 것.

        self.value_log = []
        self.trade_log = []
        self.long_win_log = []
        self.short_win_log = []

    def dayInit(self):
        self.value_today = self.ohlcv_data[self.today,3]
        self.rate_tomorrow = self.y_pred[self.today]

        self.money_total = self.money + self.amount * self.value_today

    def dayStart(self, start_date):
        self.today = start_date
        self.dayInit()
        self.value_log.append(self.value_today)
        self.trade_log.append(0)

    def append_win_log(self):
        if self.trade_log[-1] != self.trade_log[-2]:
            self.value_log.append(self.value_today)
            if self.value_log[-1] > self.value_log[-2]:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(1)
                else:
                    self.short_win_log.append(0)
            else:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(0)
                else:
                    self.short_win_log.append(1)
    
    def performAct(self):
        if self.rate_tomorrow > threshold[0]:
            buy_amount = self.money / self.value_today
            self.amount += buy_amount * (1 - self.fee_rate)
            self.money -= buy_amount * self.value_today

            self.trade_log.append(1)
            self.append_win_log()

        elif self.rate_tomorrow < threshold[1]:
            sell_amount = self.amount
            self.amount -= sell_amount
            self.money += (sell_amount * (1 - self.fee_rate)) * self.value_today

            self.trade_log.append(0)
            self.append_win_log()
        
        else:
            self.trade_log.append(self.trade_log[-1])


    def gotoTomorrow(self):
        self.today = self.today + 1
        self.dayInit()
    
    def printState(self):
        print("today: %s" % self.today)
        print("money: %s" % self.money)
        print("amount: %s" % self.amount)
        print("value_today: %s" % self.value_today)
        print("rate_tomorrow: %s" % self.rate_tomorrow)
        print("money_total: %s" % self.money_total)

In [ ]:
# 현물 지정가
class StockTrader:
    def __init__(self, predictor, X_data, y_data, ohlcv_data):
        self.today = 0
        self.money = 10000000 # 예수금
        self.amount = 0 # 보유량
        self.value_today = 0 # 오늘가
        self.value_tomorrow = 0 # 내일가
        self.rate_tomorrow = 0
        self.money_total = self.money

        self.fee_rate = 0.00
        self.fee_rate = fee_rate
        
        self.predictor = predictor # 예측모델
        self.X_data = X_data
        self.y_data = y_data
        self.ohlcv_data = ohlcv_data
        self.y_pred = self.y_data
        #self.y_pred = self.predictor.predict(self.X_data)
        self.y_pred = y_pred_global # 전역 변수
        # 0~99 까지의 rate를 통해(즉, 100의 데이터는 알고 있다.) 101의 데이터를 얻어낸 것.

        self.value_log = []
        self.trade_log = []
        self.long_win_log = []
        self.short_win_log = []

    def dayInit(self):
        self.value_today = self.ohlcv_data[self.today,3]
        self.rate_tomorrow = self.y_pred[self.today]

        self.money_total = self.money + self.amount * self.value_today

    def dayStart(self, start_date):
        self.today = start_date
        self.dayInit()
        self.value_log.append(self.value_today)
        self.trade_log.append(0)

    def append_win_log(self):
        if self.trade_log[-1] != self.trade_log[-2]:
            self.value_log.append(self.value_today)
            if self.value_log[-1] > self.value_log[-2]:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(1)
                else:
                    self.short_win_log.append(0)
            else:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(0)
                else:
                    self.short_win_log.append(1)
    
    def performAct(self):
        value_range = 0.0 / 100
        if self.rate_tomorrow > threshold[0]:
            buy_value = self.value_today * (1 - value_range)
            if self.ohlcv_data[self.today+1,2] <= buy_value:
                buy_amount = self.money / buy_value
                self.amount += buy_amount * (1 - self.fee_rate)
                self.money -= buy_amount * buy_value

                self.trade_log.append(1)
                self.append_win_log()

        elif self.rate_tomorrow < threshold[1]:
            sell_value = self.value_today * (1 + value_range)
            if self.ohlcv_data[self.today+1,1] >= sell_value:
                sell_amount = self.amount
                self.amount -= sell_amount
                self.money += (sell_amount * (1 - self.fee_rate)) * sell_value

                self.trade_log.append(0)
                self.append_win_log()
        
        else:
            self.trade_log.append(self.trade_log[-1])


    def gotoTomorrow(self):
        self.today = self.today + 1
        self.dayInit()
    
    def printState(self):
        print("today: %s" % self.today)
        print("money: %s" % self.money)
        print("amount: %s" % self.amount)
        print("value_today: %s" % self.value_today)
        print("rate_tomorrow: %s" % self.rate_tomorrow)
        print("money_total: %s" % self.money_total)

In [ ]:
# 레버리지
class StockTrader:
    def __init__(self, predictor, X_data, y_data, ohlcv_data):
        self.today = 0
        self.money = 10000000 # 예수금
        self.amounts = [0,0] # 보유량
        self.value_today = 0
        self.values_today = [1,1] # 오늘가
        self.value_tomorrow = 0 # 내일가
        self.rate_tomorrow = 0
        self.money_total = self.money

        self.fee_rate = 0.001
        self.fee_rate = fee_rate
        
        self.predictor = predictor # 예측모델
        self.X_data = X_data
        self.y_data = y_data
        self.ohlcv_data = ohlcv_data
        self.y_pred = self.y_data
        #self.y_pred = self.predictor.predict(self.X_data)
        self.y_pred = y_pred_global # 전역 변수
        # 0~99 까지의 rate를 통해(즉, 100의 데이터는 알고 있다.) 101의 데이터를 얻어낸 것.

        self.value_log = []
        self.trade_log = []
        self.long_win_log = []
        self.short_win_log = []

    def dayInit(self):
        global reverlage_rate
        self.rate_today = self.ohlcv_data[self.today,0] / self.ohlcv_data[self.today-1,0] 
        
        self.value_today = self.ohlcv_data[self.today,0]
        self.values_today[0] *= (self.rate_today-1) * reverlage_rate + 1
        self.values_today[1] *= (1-self.rate_today) * reverlage_rate + 1

        self.rate_tomorrow = self.y_pred[self.today]

        self.money_total = self.money + self.amounts[0] * self.values_today[0] + self.amounts[1] * self.values_today[1]

    def dayStart(self, start_date):
        self.today = start_date
        self.dayInit()
        self.value_log.append(self.value_today)
        self.trade_log.append(-1)    

    def append_win_log(self):
        if self.trade_log[-1] != self.trade_log[-2]:
            self.value_log.append(self.value_today)
            if self.value_log[-1] > self.value_log[-2]:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(1)
                else:
                    self.short_win_log.append(0)
            else:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(0)
                else:
                    self.short_win_log.append(1)
    
    def performAct(self):
        if self.rate_tomorrow > threshold[0]:
            self.money += self.amounts[1] * self.values_today[1] * (1 - self.fee_rate)
            self.amounts[1] = 0

            buy_amount = self.money / self.values_today[0]
            self.amounts[0] += buy_amount * (1 - self.fee_rate)
            self.money -= buy_amount * self.values_today[0]

            self.trade_log.append(1)
            self.append_win_log()
            
        elif self.rate_tomorrow < threshold[1]:
            self.money += self.amounts[0] * self.values_today[0] * (1 - self.fee_rate)
            self.amounts[0] = 0

            buy_amount = self.money / self.values_today[1]
            self.amounts[1] += buy_amount * (1 - self.fee_rate)
            self.money -= buy_amount * self.values_today[1]

            self.trade_log.append(-1)
            self.append_win_log()
        
        else:
            self.trade_log.append(self.trade_log[-1])
            


    def gotoTomorrow(self):
        self.today = self.today + 1
        self.dayInit()
    
    def printState(self):
        print("today: %s" % self.today)
        print("money: %s" % self.money)
        print("amount: %s" % self.amount)
        print("value_today: %s" % self.value_today)
        print("rate_tomorrow: %s" % self.rate_tomorrow)
        print("money_total: %s" % self.money_total)

In [ ]:
# 레버리지 분할 매매
class StockTrader:
    def __init__(self, predictor, X_data, y_data, ohlcv_data):
        self.today = 0
        self.money = 10000000 # 예수금
        self.amounts = [0,0] # 보유량
        self.value_today = 0
        self.values_today = [1,1] # 오늘가
        self.value_tomorrow = 0 # 내일가
        self.rate_tomorrow = 0
        self.money_total = self.money

        self.fee_rate = 0.001
        self.fee_rate = fee_rate
        
        self.predictor = predictor # 예측모델
        self.X_data = X_data
        self.y_data = y_data
        self.ohlcv_data = ohlcv_data
        self.value_data = ohlcv_data[:,3]
        self.y_pred = self.y_data
        #self.y_pred = self.predictor.predict(self.X_data)
        self.y_pred = y_pred_global # 전역 변수
        # 0~99 까지의 rate를 통해(즉, 100의 데이터는 알고 있다.) 101의 데이터를 얻어낸 것.

        self.value_log = []
        self.trade_log = []
        self.long_win_log = []
        self.short_win_log = []

    def dayInit(self):
        global reverlage_rate
        self.rate_today = self.value_data[self.today] / self.value_data[self.today-1] 
        
        self.value_today = self.value_data[self.today]
        self.values_today[0] *= (self.rate_today-1) * reverlage_rate + 1
        self.values_today[1] *= (1-self.rate_today) * reverlage_rate + 1

        self.rate_tomorrow = self.y_pred[self.today]

        self.money_total = self.money + self.amounts[0] * self.values_today[0] + self.amounts[1] * self.values_today[1]

    def dayStart(self, start_date):
        self.today = start_date
        self.dayInit()
        self.value_log.append(self.value_today)
        self.trade_log.append(0)    

    def append_win_log(self):
        if self.trade_log[-1] != self.trade_log[-2]:
            self.value_log.append(self.value_today)
            if self.value_log[-1] > self.value_log[-2]:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(1)
                elif self.trade_log[-2] == -1:
                    self.short_win_log.append(0)
            else:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(0)
                elif self.trade_log[-2] == -1:
                    self.short_win_log.append(1)
    
    def performAct(self):
        if self.rate_tomorrow > threshold[0]:
            self.money += self.amounts[1] * self.values_today[1] * (1 - self.fee_rate) / 2
            self.amounts[1] = self.amounts[1] * 1 / 2

            buy_amount = self.money / self.values_today[0]
            self.amounts[0] += buy_amount * (1 - self.fee_rate)
            self.money -= buy_amount * self.values_today[0]

            self.trade_log.append(1)
            self.append_win_log()
            
        elif self.rate_tomorrow < threshold[1]:
            self.money += self.amounts[0] * self.values_today[0] * (1 - self.fee_rate) / 2
            self.amounts[0] = self.amounts[0] * 1 / 2

            buy_amount = self.money / self.values_today[1]
            self.amounts[1] += buy_amount * (1 - self.fee_rate)
            self.money -= buy_amount * self.values_today[1]

            self.trade_log.append(-1)
            self.append_win_log()
        
        elif self.rate_tomorrow > threshold[0] / 2:
            self.money += self.amounts[1] * self.values_today[1] * (1 - self.fee_rate) / 4
            self.amounts[1] = self.amounts[1] * 3 / 4

            buy_amount = self.money / self.values_today[0]
            self.amounts[0] += buy_amount * (1 - self.fee_rate)
            self.money -= buy_amount * self.values_today[0]

            self.trade_log.append(1)
            self.append_win_log()
            
        elif self.rate_tomorrow < threshold[1] / 2:
            self.money += self.amounts[0] * self.values_today[0] * (1 - self.fee_rate) / 4
            self.amounts[0] = self.amounts[0] * 3 / 4

            buy_amount = self.money / self.values_today[1]
            self.amounts[1] += buy_amount * (1 - self.fee_rate)
            self.money -= buy_amount * self.values_today[1]

            self.trade_log.append(-1)
            self.append_win_log()
        
        else:
            self.trade_log.append(self.trade_log[-1])
            


    def gotoTomorrow(self):
        self.today = self.today + 1
        self.dayInit()
    
    def printState(self):
        print("today: %s" % self.today)
        print("money: %s" % self.money)
        print("amount: %s" % self.amount)
        print("value_today: %s" % self.value_today)
        print("rate_tomorrow: %s" % self.rate_tomorrow)
        print("money_total: %s" % self.money_total)

In [ ]:
# 레버리지 손익절
class StockTrader:
    def __init__(self, predictor, X_data, y_data, ohlcv_data):
        self.today = 0
        self.money = 10000000 # 예수금
        self.amounts = [0,0] # 보유량
        self.value_today = 0
        self.values_today = [1,1] # 오늘가
        self.value_tomorrow = 0 # 내일가
        self.rate_tomorrow = 0
        self.money_total = self.money

        self.fee_rate = 0.001
        self.fee_rate = fee_rate
        
        self.predictor = predictor # 예측모델
        self.X_data = X_data
        self.y_data = y_data
        self.ohlcv_data = ohlcv_data
        self.value_data = ohlcv_data
        self.y_pred = self.y_data
        #self.y_pred = self.predictor.predict(self.X_data)
        self.y_pred = y_pred_global # 전역 변수
        # 0~99 까지의 rate를 통해(즉, 100의 데이터는 알고 있다.) 101의 데이터를 얻어낸 것.

        self.value_log = []
        self.trade_log = []
        self.long_win_log = []
        self.short_win_log = []

        self.flag = 0

    def dayInit(self):
        reverlage_rate = 2
        self.rate_today = self.value_data[self.today] / self.value_data[self.today-1] 
        
        self.value_today = self.value_data[self.today]
        self.values_today[0] *= (self.rate_today-1) * reverlage_rate + 1
        self.values_today[1] *= (1-self.rate_today) * reverlage_rate + 1

        self.rate_tomorrow = self.y_pred[self.today]

        self.money_total = self.money + self.amounts[0] * self.values_today[0] + self.amounts[1] * self.values_today[1]

    def dayStart(self, start_date):
        self.today = start_date
        self.dayInit()
        self.value_log.append(self.value_today)
        self.trade_log.append(0)    

    def append_win_log(self):
        if self.trade_log[-1] != self.trade_log[-2]:
            self.value_log.append(self.value_today)
            if self.value_log[-1] > self.value_log[-2]:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(1)
                elif self.trade_log[-2] == -1:
                    self.short_win_log.append(0)
            else:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(0)
                elif self.trade_log[-2] == -1:
                    self.short_win_log.append(1)
    
    def performAct(self):
        if self.flag == 0:
            if self.rate_tomorrow > threshold[0]:
                self.money += self.amounts[1] * self.values_today[1] * (1 - self.fee_rate)
                self.amounts[1] = 0

                buy_amount = self.money / self.values_today[0]
                self.amounts[0] += buy_amount * (1 - self.fee_rate)
                self.money -= buy_amount * self.values_today[0]

                self.trade_log.append(1)
                self.append_win_log()

                self.flag = 1
                
            elif self.rate_tomorrow < threshold[1]:
                self.money += self.amounts[0] * self.values_today[0] * (1 - self.fee_rate)
                self.amounts[0] = 0

                buy_amount = self.money / self.values_today[1]
                self.amounts[1] += buy_amount * (1 - self.fee_rate)
                self.money -= buy_amount * self.values_today[1]

                self.trade_log.append(-1)
                self.append_win_log()

                self.flag = -1

        elif self.flag == 1:
            if self.rate_tomorrow < threshold[0]:
                if self.value_today / self.value_log[-1] > (1+rate_threshold) or self.value_today / self.value_log[-1] < (1-rate_threshold):
                    self.money += self.amounts[0] * self.values_today[0] * (1 - self.fee_rate)
                    self.amounts[0] = 0

                    self.trade_log.append(0)
                    self.append_win_log()

                    self.flag = 0
        
        elif self.flag == -1:
            if self.rate_tomorrow > threshold[1]:
                if self.value_today / self.value_log[-1] > (1+rate_threshold) or self.value_today / self.value_log[-1] < (1-rate_threshold):
                    self.money += self.amounts[1] * self.values_today[1] * (1 - self.fee_rate)
                    self.amounts[1] = 0

                    self.trade_log.append(0)
                    self.append_win_log()

                    self.flag = 0
            

        else:
            self.trade_log.append(self.trade_log[-1])
            


    def gotoTomorrow(self):
        self.today = self.today + 1
        self.dayInit()
    
    def printState(self):
        print("today: %s" % self.today)
        print("money: %s" % self.money)
        print("amount: %s" % self.amount)
        print("value_today: %s" % self.value_today)
        print("rate_tomorrow: %s" % self.rate_tomorrow)
        print("money_total: %s" % self.money_total)

In [ ]:
# 선물
class StockTrader:
    def __init__(self, predictor, X_data, y_data, ohlcv_data):
        self.today = 0
        self.money = 10000000 # 예수금
        self.amounts = [0,0] #/ 보유량
        self.value_today = 0
        self.values_today = [1,1] # 오늘가
        self.value_tomorrow = 0 # 내일가
        self.rate_tomorrow = 0
        self.money_total = self.money
        self.dept = 0

        self.fee_rate = 0.001
        self.fee_rate = fee_rate
        
        self.predictor = predictor # 예측모델
        self.X_data = X_data
        self.y_data = y_data
        self.ohlcv_data = ohlcv_data
        self.y_pred = self.y_data
        #self.y_pred = self.predictor.predict(self.X_data)
        self.y_pred = y_pred_global # 전역 변수
        # 0~99 까지의 rate를 통해(즉, 100의 데이터는 알고 있다.) 101의 데이터를 얻어낸 것.

        self.value_log = []
        self.trade_log = []
        self.long_win_log = []
        self.short_win_log = []

    def dayInit(self):
        reverlage_rate = 1
        self.rate_today = self.ohlcv_data[self.today,0] / self.ohlcv_data[self.today-1,0] 
        
        self.value_today = self.ohlcv_data[self.today,0]
        self.values_today[0] *= (self.rate_today-1) * reverlage_rate + 1
        self.values_today[1] *= (1-self.rate_today) * reverlage_rate + 1

        self.rate_tomorrow = self.y_pred[self.today]

        self.money_total = self.money + self.amounts[0] * self.values_today[0] + self.amounts[1] * self.values_today[1] - self.dept
        if self.money_total < 0:
            self.money = 0; self.dept = 0
            self.amounts[0] = 0; self.amounts[1] = 0
            self.money_total = 0

    def dayStart(self, start_date):
        self.today = start_date
        self.dayInit()
        self.value_log.append(self.value_today)
        self.trade_log.append(0)    

    def append_win_log(self):
        if self.trade_log[-1] != self.trade_log[-2]:
            self.value_log.append(self.value_today)
            if self.value_log[-1] > self.value_log[-2]:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(1)
                elif self.trade_log[-2] == -1:
                    self.short_win_log.append(0)
            else:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(0)
                elif self.trade_log[-2] == -1:
                    self.short_win_log.append(1)
    
    def controlPosition(self, position_rates):
        global margin_rate
        current_rates = []
        for i_position in range(len(self.amounts)): # 포지션 확인
            current_rates.append(self.amounts[i_position] * self.values_today[i_position] / self.money_total)

        for i_position in range(len(self.amounts)): # 포지션 매도
            if position_rates[i_position] * margin_rate < current_rates[i_position]:
                diff_rate = current_rates[i_position] - position_rates[i_position] * margin_rate
                sell_amount = self.money_total / self.values_today[i_position] * diff_rate
                self.money += sell_amount * self.values_today[i_position] * (1 - self.fee_rate)
                self.amounts[i_position] -= sell_amount

        if margin_rate > 1: # 선물 차용
            self.money -= self.dept
            self.dept = self.money * (margin_rate - 1)
            self.money += self.dept

        for i_position in range(len(self.amounts)): # 포지션 매수
            if position_rates[i_position] * margin_rate > current_rates[i_position]:
                diff_rate = position_rates[i_position] * margin_rate - current_rates[i_position]
                buy_amount = self.money_total / self.values_today[i_position] * diff_rate
                self.amounts[i_position] += buy_amount * (1 - self.fee_rate)
                self.money -= buy_amount * self.values_today[i_position]
        
        self.money_total = self.money + self.amounts[0] * self.values_today[0] + self.amounts[1] * self.values_today[1] - self.dept


    def performAct(self):
        if self.rate_tomorrow > threshold[0]:
            self.controlPosition([1.0, 0.0])

            self.trade_log.append(1)
            self.append_win_log()
            
        elif self.rate_tomorrow < threshold[1]:
            self.controlPosition([0.0, 1.0])

            self.trade_log.append(-1)
            self.append_win_log()

        else:
            self.trade_log.append(self.trade_log[-1])


    def gotoTomorrow(self):
        self.today = self.today + 1
        self.dayInit()
    
    def printState(self):
        print("today: %s" % self.today)
        print("money: %s" % self.money)
        print("amount: %s" % self.amount)
        print("value_today: %s" % self.value_today)
        print("rate_tomorrow: %s" % self.rate_tomorrow)
        print("money_total: %s" % self.money_total)

In [ ]:
# 선물 실제
class StockTrader:
    def __init__(self, ohlcv_data, y_pred):
        self.today = 0
        self.money = 10000000 # 예수금
        self.amount = 0 # 보유량
        self.value_today = 0
        self.y_today = 0
        self.money_total = self.money

        global margin_rate
        self.margin_rate = margin_rate
        global fee_rate
        self.fee_rate = fee_rate

        self.ohlcv_data = ohlcv_data
        self.y_pred = y_pred

        global threshold
        self.threshold = threshold

        self.value_log = []
        self.trade_log = []
        self.long_win_log = []
        self.short_win_log = []

        self.position_rate = 0

    def dayInit(self):
        self.value_today = self.ohlcv_data[self.today,3]
        self.y_today = self.y_pred[self.today]

        self.money_total = self.money + self.amount * self.value_today
        if self.money_total < 0:
            self.money = 0;
            self.amount = 0
            self.money_total = 0

    def dayStart(self, start_date):
        self.today = start_date
        self.dayInit()
        self.value_log.append(self.value_today)
        self.trade_log.append(0)    

    def append_win_log(self):
        if self.trade_log[-1] != self.trade_log[-2]:
            self.value_log.append(self.value_today)
            if self.value_log[-1] > self.value_log[-2]:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(1)
                elif self.trade_log[-2] == -1:
                    self.short_win_log.append(0)
            else:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(0)
                elif self.trade_log[-2] == -1:
                    self.short_win_log.append(1)

    def buyAmount(self, amount_rate):
        self.money_total = self.money + self.amount * self.value_today
        available_money = self.money_total * self.margin_rate - self.amount * self.value_today

        if available_money > self.money_total * self.margin_rate * 0.05:
            #print("BUY: " + str(self.value_today))
            buy_amount = available_money / self.value_today * amount_rate
            self.money -= buy_amount * self.value_today
            self.amount += buy_amount * (1 - self.fee_rate)

    def sellAmount(self, amount_rate):
        self.money_total = self.money + self.amount * self.value_today
        available_money = self.money_total * self.margin_rate + self.amount * self.value_today

        if available_money > self.money_total * self.margin_rate * 0.05:
            #print("SELL: " + str(self.value_today))
            sell_amount = available_money / self.value_today * amount_rate
            self.money += sell_amount * self.value_today * (1 - self.fee_rate)
            self.amount -= sell_amount

    def performAct(self):
        if self.amount * self.value_today > self.money_total * (self.margin_rate * 2.0):
            self.sellAmount(0.5)
        elif -1 * self.amount * self.value_today > self.money_total * (self.margin_rate * 2.0):
            self.buyAmount(0.5)

        if self.y_today > self.threshold[0]:
            self.buyAmount(1.0)

            self.trade_log.append(1)
            self.append_win_log()
            
        elif self.y_today < self.threshold[1]:
            self.sellAmount(1.0)

            self.trade_log.append(-1)
            self.append_win_log()

        else:
            self.trade_log.append(self.trade_log[-1])


    def gotoTomorrow(self):
        self.today = self.today + 1
        self.dayInit()
    
    def printState(self):
        print("today: %s" % self.today)
        print("money: %s" % self.money)
        print("amount: %s" % self.amount)
        print("value_today: %s" % self.value_today)
        print("money_total: %s" % self.money_total)

In [ ]:
# 선물 실제 + 지정가
class StockTrader:
    def __init__(self, ohlcv_data, y_pred):
        self.today = 0
        self.money = 10000000 # 예수금
        self.amount = 0 # 보유량
        self.value_today = 0
        self.y_today = 0
        self.money_total = self.money

        global margin_rate
        self.margin_rate = margin_rate
        global fee_rate
        self.fee_rate = fee_rate

        self.ohlcv_data = ohlcv_data
        self.y_pred = y_pred

        global threshold
        self.threshold = threshold

        global limit_rate
        self.limit_rate = limit_rate

        self.value_log = []
        self.trade_log = []
        self.long_win_log = []
        self.short_win_log = []

        self.position_rate = 0

    def dayInit(self):
        self.value_today = self.ohlcv_data[self.today,3]
        self.y_today = self.y_pred[self.today]

        self.money_total = self.money + self.amount * self.value_today
        if self.money_total < 0:
            self.money = 0
            self.amount = 0
            self.money_total = 0

    def dayStart(self, start_date):
        self.today = start_date
        self.dayInit()
        self.value_log.append(self.value_today)
        self.trade_log.append(0)

    def append_win_log(self, log):
        self.trade_log.append(log)
        self.value_log.append(self.limit_value)
        if self.value_log[-1] > self.value_log[-2]:
            if self.trade_log[-2] == 1:
                self.long_win_log.append(1)
            elif self.trade_log[-2] == -1:
                self.short_win_log.append(0)
        else:
            if self.trade_log[-2] == 1:
                self.long_win_log.append(0)
            elif self.trade_log[-2] == -1:
                self.short_win_log.append(1)

    def buyAmount(self, amount_rate):
        if self.amount <= 0:
            self.low_tomorrow = self.ohlcv_data[self.today+1, 2]
            self.limit_value = self.value_today * (1 - self.limit_rate)
            self.money_total = self.money + self.amount * self.limit_value
            available_money = self.money_total * self.margin_rate - self.amount * self.limit_value
            if self.limit_value >= self.low_tomorrow and available_money > self.money_total * self.margin_rate * 0.1:
                buy_amount = available_money / self.limit_value * amount_rate
                #print("BUY: " + str(self.limit_value), buy_amount)
                self.money -= buy_amount * self.limit_value
                self.amount += buy_amount * (1 - self.fee_rate)

                self.append_win_log(1)

    def sellAmount(self, amount_rate):
        if self.amount >= 0:
            self.high_tomorrow = self.ohlcv_data[self.today+1, 1]
            self.limit_value = self.value_today * (1 + self.limit_rate)
            self.money_total = self.money + self.amount * self.limit_value
            available_money = self.money_total * self.margin_rate + self.amount * self.limit_value
            if self.limit_value <= self.high_tomorrow and available_money > self.money_total * self.margin_rate * 0.1:
                sell_amount = available_money / self.limit_value * amount_rate
                #print("SELL: " + str(self.limit_value), sell_amount)
                self.money += sell_amount * self.limit_value * (1 - self.fee_rate)
                self.amount -= sell_amount

                self.append_win_log(-1)
    
    def closeAmount(self):
        overflow_money = abs(self.amount) * self.value_today - (self.money_total * self.margin_rate)
        close_amount = overflow_money / self.value_today * self.amount / abs(self.amount)
        self.money += close_amount * self.value_today * (1 - 0.04 / 100)
        self.amount -= close_amount
        self.money_total = self.money + self.amount * self.value_today


    def performAct(self):
        risk_rate = self.amount * self.value_today / (self.money_total * self.margin_rate)
        #if abs(risk_rate) > 2:
        #    self.closeAmount()
        
        if risk_rate > 2:
            self.sellAmount(0.5)
        elif risk_rate < -2:
            self.buyAmount(0.5)
        
        elif self.y_today > self.threshold[0]:
            self.buyAmount(1.0)

        elif self.y_today < self.threshold[1]:
            self.sellAmount(1.0)
        
        self.money_total = self.money + self.amount * self.value_today


    def gotoTomorrow(self):
        self.today = self.today + 1
        self.dayInit()

    def printState(self):
        print("today: %s" % self.today)
        print("money: %s" % self.money)
        print("amount: %s" % self.amount)
        print("value_today: %s" % self.value_today)
        print("money_total: %s" % self.money_total)

In [ ]:
# 선물 실제 + 지정가 + 가중치
class StockTrader:
    def __init__(self, ohlcv_data, y_pred):
        self.today = 0
        self.money = 10000000 # 예수금
        self.amount = 0 # 보유량
        self.value_today = 0
        self.y_today = 0
        self.money_total = self.money

        global margin_rate
        self.margin_rate = margin_rate
        global fee_rate
        self.fee_rate = fee_rate

        self.ohlcv_data = ohlcv_data
        self.y_pred = y_pred

        global threshold
        self.threshold = threshold

        global limit_rate
        self.limit_rate_origin = limit_rate
        self.limit_rate = limit_rate

        self.value_log = []
        self.trade_log = []
        self.long_win_log = []
        self.short_win_log = []

        self.position_rate = 0

    def dayInit(self):
        self.value_today = self.ohlcv_data[self.today,3]
        self.y_today = self.y_pred[self.today]
        self.limit_rate = (1 - np.clip(np.abs(self.y_today), 0, 1)) * self.limit_rate_origin

        self.money_total = self.money + self.amount * self.value_today
        if self.money_total < 0:
            self.money = 0
            self.amount = 0
            self.money_total = 0

    def dayStart(self, start_date):
        self.today = start_date
        self.dayInit()
        self.value_log.append(self.value_today)
        self.trade_log.append(0)

    def append_win_log(self, log):
        self.trade_log.append(log)
        self.value_log.append(self.limit_value)
        if self.value_log[-1] > self.value_log[-2]:
            if self.trade_log[-2] == 1:
                self.long_win_log.append(1)
            elif self.trade_log[-2] == -1:
                self.short_win_log.append(0)
        else:
            if self.trade_log[-2] == 1:
                self.long_win_log.append(0)
            elif self.trade_log[-2] == -1:
                self.short_win_log.append(1)

    def buyAmount(self, amount_rate):
        if self.amount <= 0:
            self.low_tomorrow = self.ohlcv_data[self.today+1, 2]
            self.limit_value = self.value_today * (1 - self.limit_rate)
            self.money_total = self.money + self.amount * self.limit_value
            available_money = self.money_total * self.margin_rate - self.amount * self.limit_value
            if self.limit_value >= self.low_tomorrow and available_money > self.money_total * self.margin_rate * 0.1:
                buy_amount = available_money / self.limit_value * amount_rate
                #print("BUY: " + str(self.limit_value), buy_amount)
                self.money -= buy_amount * self.limit_value
                self.amount += buy_amount * (1 - self.fee_rate)

                self.append_win_log(1)

    def sellAmount(self, amount_rate):
        if self.amount >= 0:
            self.high_tomorrow = self.ohlcv_data[self.today+1, 1]
            self.limit_value = self.value_today * (1 + self.limit_rate)
            self.money_total = self.money + self.amount * self.limit_value
            available_money = self.money_total * self.margin_rate + self.amount * self.limit_value
            if self.limit_value <= self.high_tomorrow and available_money > self.money_total * self.margin_rate * 0.1:
                sell_amount = available_money / self.limit_value * amount_rate
                #print("SELL: " + str(self.limit_value), sell_amount)
                self.money += sell_amount * self.limit_value * (1 - self.fee_rate)
                self.amount -= sell_amount

                self.append_win_log(-1)
    
    def closeAmount(self):
        overflow_money = abs(self.amount) * self.value_today - (self.money_total * self.margin_rate)
        close_amount = overflow_money / self.value_today * self.amount / abs(self.amount)
        self.money += close_amount * self.value_today * (1 - 0.04 / 100)
        self.amount -= close_amount
        self.money_total = self.money + self.amount * self.value_today


    def performAct(self):
        risk_rate = self.amount * self.value_today / (self.money_total * self.margin_rate)
        #if abs(risk_rate) > 2:
        #    self.closeAmount()
        
        if risk_rate > 2:
            self.sellAmount(0.5)
        elif risk_rate < -2:
            self.buyAmount(0.5)
        
        elif self.y_today > self.threshold[0]:
            self.buyAmount(1.0)

        elif self.y_today < self.threshold[1]:
            self.sellAmount(1.0)
        
        self.money_total = self.money + self.amount * self.value_today


    def gotoTomorrow(self):
        self.today = self.today + 1
        self.dayInit()

    def printState(self):
        print("today: %s" % self.today)
        print("money: %s" % self.money)
        print("amount: %s" % self.amount)
        print("value_today: %s" % self.value_today)
        print("money_total: %s" % self.money_total)

In [ ]:
# 선물 실제 분할 매매
class StockTrader:
    def __init__(self, ohlcv_data, y_pred):
        self.today = 0
        self.money = 10000000 # 예수금
        self.amount = 0 #/ 보유량
        self.value_today = 0
        self.y_today = 0
        self.money_total = self.money

        global margin_rate
        self.margin_rate = margin_rate
        global fee_rate
        self.fee_rate = fee_rate
        
        self.ohlcv_data = ohlcv_data
        self.y_pred = y_pred

        self.value_log = []
        self.trade_log = []
        self.long_win_log = []
        self.short_win_log = []

    def dayInit(self):
        self.value_today = self.ohlcv_data[self.today,3]
        self.y_today = self.y_pred[self.today]

        self.money_total = self.money + self.amount * self.value_today
        if self.money_total < 0:
            self.money = 0;
            self.amount = 0
            self.money_total = 0

    def dayStart(self, start_date):
        self.today = start_date
        self.dayInit()
        self.value_log.append(self.value_today)
        self.trade_log.append(0)    

    def append_win_log(self):
        if self.trade_log[-1] != self.trade_log[-2]:
            self.value_log.append(self.value_today)
            if self.value_log[-1] > self.value_log[-2]:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(1)
                elif self.trade_log[-2] == -1:
                    self.short_win_log.append(0)
            else:
                if self.trade_log[-2] == 1:
                    self.long_win_log.append(0)
                elif self.trade_log[-2] == -1:
                    self.short_win_log.append(1)
    
    def buyAmount(self, amount_rate):
        self.money_total = self.money + self.amount * self.value_today
        available_money = self.money_total * self.margin_rate - self.amount * self.value_today
        
        buy_amount = available_money / self.value_today * amount_rate
        self.money -= buy_amount * self.value_today
        self.amount += buy_amount * (1 - self.fee_rate)

    def sellAmount(self, amount_rate):
        self.money_total = self.money + self.amount * self.value_today
        available_money = self.money_total * self.margin_rate + self.amount * self.value_today
        
        sell_amount = available_money / self.value_today * amount_rate
        self.money += sell_amount * self.value_today * (1 - self.fee_rate)
        self.amount -= sell_amount

    def performAct(self):
        if self.y_today > threshold[0]:
            self.buyAmount(0.5)

            self.trade_log.append(1)
            self.append_win_log()
        
        elif self.y_today > threshold[0] * 0.5:
            self.buyAmount(0.25)

            self.trade_log.append(1)
            self.append_win_log()
            
        elif self.y_today < threshold[1]:
            self.sellAmount(0.5)

            self.trade_log.append(-1)
            self.append_win_log()
        
        elif self.y_today < threshold[1] * 0.5:
            self.sellAmount(0.25)

            self.trade_log.append(-1)
            self.append_win_log()

        else:
            self.trade_log.append(self.trade_log[-1])


    def gotoTomorrow(self):
        self.today = self.today + 1
        self.dayInit()
    
    def printState(self):
        print("today: %s" % self.today)
        print("money: %s" % self.money)
        print("amount: %s" % self.amount)
        print("value_today: %s" % self.value_today)
        print("money_total: %s" % self.money_total)

In [ ]:
def getYield(ohlcv_data, y_pred):
    period = y_pred.shape[0]
    trader = StockTrader(ohlcv_data, y_pred)

    trader.dayStart(0)

    money_list = []
    value_list = []

    for i in range(period - 0): # 마지막 1번의 예측 미수행(y가 없으므로)
        trader.dayInit()
        trader.performAct()
        money_list.append(trader.money_total)
        value_list.append(trader.value_today)
        trader.today += 1
        #trader.printState()
    '''
    plt.clf()
    #plt.plot(list(range(len(win_list))), win_list)
    plt.plot(list(range(len(value_list))), value_list/value_list[0])
    plt.plot(list(range(len(money_list))), money_list/money_list[0])
    plt.show()
    '''
    money_list = np.array(money_list)
    value_list = np.array(value_list)
    trade_num = [len(trader.long_win_log), len(trader.short_win_log)]
    win_rate = [np.mean(trader.long_win_log) * 100, np.mean(trader.short_win_log) * 100]
    trading_yield = money_list[len(money_list)-1]/money_list[0] * 100

    trading_info = [trader.trade_log, trade_num, win_rate, trading_yield]

    return trading_info, money_list, value_list

In [ ]:
ohlcv_data = test_datas["DOGE"]
data_start = 0
data_end = ohlcv_data.shape[0] - start_diff - 1
period = data_end - data_start

predict = True
#predict = False
if predict:
    y_preds = np.zeros((0,) + y_shape)
    y_datas = np.zeros((0,) + y_shape)
    period_start = data_start
    div_period = 1024
    while(period_start < data_end):
        min_ix = min(period_start+div_period, data_end)
        ix = np.arange(period_start, min_ix)

        X_data = np.zeros(((len(ix),) + X_shape))
        y_data = np.zeros(((len(ix),) + y_shape))
        get_Xy(ohlcv_data[:,3], ix+start_diff, X_data, y_data, train=False)
        
        y_pred = y_data[:len(ix)]
        y_pred = predictor.predict(X_data[:len(ix)], verbose=0)

        y_preds = np.append(y_preds, y_pred, axis=0)
        y_datas = np.append(y_datas, y_data, axis=0)

        period_start = period_start + len(ix)

y_pred = y_preds
y_data = y_datas

margin_rate = 1
reverlage_rate = 1
th = 0.0
threshold = [th, -th]
fee_rate = 0.0 / 100
limit_rate = 0.0 / 100

plt.clf()
plt.plot(list(range(len(y_data))), y_data)
plt.plot(list(range(len(y_pred))), y_pred)
plt.show()

threshold_line = np.zeros((y_pred.shape[0], len(threshold)))
threshold_line[:,0] = threshold[0]
threshold_line[:,1] = threshold[1]
plt.clf()
plt.plot(list(range(len(y_pred))), y_pred)
plt.plot(list(range(len(threshold_line))), threshold_line)
plt.show()

trading_info, money_list, value_list = getYield(ohlcv_data[data_start+start_diff:], y_pred)
# trade_log, trade_num, win_rate, trading_yield
print("거래횟수: %s, %s 회" % (trading_info[1][0], trading_info[1][1]))
print("승률: %s, %s %%" % (trading_info[2][0], trading_info[2][1]))
trading_yield = (money_list[-1]/money_list[0] - 1) * 100
print("수익률: %s %%" % trading_yield)

plt.clf()
plt.title("value and asset")
plt.xlabel("time(minute)")
plt.ylabel("scale")
plt.plot(list(range(len(value_list))), value_list/value_list[0])
plt.plot(list(range(len(money_list))), money_list/money_list[0])

trade_log = np.array(trading_info[0]) * (np.max(value_list)/value_list[0] - np.min(value_list)/value_list[0]) + np.min(value_list)/value_list[0]
plt.show()
print("Last y_pred: ", y_preds[-1])

In [ ]:
yields = []
yields_origin = []
period = int(len(value_list)/10)
for i in range(10):
    start = period * i
    plt.clf()
    plt.plot(list(range(period)), value_list[start:start+period]/value_list[start])
    plt.plot(list(range(period)), money_list[start:start+period]/money_list[start])
    plt.show()
    yields.append(money_list[start+period-1]/money_list[start])
    yields_origin.append(value_list[start+period-1]/value_list[start])
print(yields)
print(yields_origin)

In [ ]:
max_money = money_list[0]
period = 3000
money_diff = []
for i_batch in range(len(money_list)-period):
    max_money = money_list[i_batch]
    min_money = min(money_list[i_batch:i_batch+period])
    money_diff.append(1-min_money/max_money)

print(max(money_diff)*100) # 시작점 대비 최대 손실율
print(np.mean(money_diff)*100) # 평균 손실율

In [ ]:
# threshold 최적화
yields = []
win_rates = []

margin_rate = 1
fee_rate = 0.04 / 100
limit_rate = 0.0 / 100
mul_threshold = 1.0 / 100

for i in range(100):
    threshold = [0.0 + mul_threshold * i, 0.0 - mul_threshold * i]
    trading_info, money_list, value_list = getYield(ohlcv_data[data_start+start_diff:], y_pred)
    win_rates.append(trading_info[2])
    yields.append((money_list[-1]/money_list[0] - 1) * 100)

    plt.clf()
    plt.plot(list(range(len(value_list))), value_list/value_list[0])
    plt.plot(list(range(len(money_list))), money_list/money_list[0])
    plt.show()

In [ ]:
# threshold 최대 그래프 출력
max_idx = np.argmax(yields) 
max_yield = yields[max_idx]

threshold = [0.0 + mul_threshold * max_idx, 0.0 - mul_threshold * max_idx]
trading_info, money_list, value_list = getYield(ohlcv_data[data_start+start_diff:], y_pred)

print("거래횟수: %s, %s 회" % (trading_info[1][0], trading_info[1][1]))
print("승률: %s, %s %%" % (trading_info[2][0], trading_info[2][1]))
trading_yield = (money_list[-1]/money_list[0] - 1) * 100
print("수익률: %s %%" % trading_yield)

plt.clf()
plt.plot(list(range(len(value_list))), value_list/value_list[0])
plt.plot(list(range(len(money_list))), money_list/money_list[0])
plt.show()

print("max_idx, max_yield: ", max_idx, max_yield)
print("threshold: ", threshold)
print("limit_rate: ", limit_rate * 100)

plt.clf()
plt.plot(list(range(len(yields))), yields)
plt.show()

plt.clf()
plt.plot(list(range(len(win_rates))), win_rates)
plt.show()

In [ ]:
# limit_rate 최적화
yields = []
win_rates = []

margin_rate = 1
fee_rate = 0.02 / 100
threshold = [0.0, -0.0]
mul_limit_rate = 1.0 / 100

for i in range(100):
    y_pred = y_preds[:,0]
    limit_rate = mul_limit_rate * i / 100
    trading_info, money_list, value_list = getYield(ohlcv_data[data_start+start_diff:], y_pred)
    win_rates.append(trading_info[2])
    yields.append((money_list[-1]/money_list[0] - 1) * 100)

    plt.clf()
    plt.plot(list(range(len(value_list))), value_list/value_list[0])
    plt.plot(list(range(len(money_list))), money_list/money_list[0])
    plt.show()

In [ ]:
# limit rate 최대 그래프 출력
max_idx = np.argmax(yields[1:]) + 1
max_yield = yields[max_idx]

threshold = [0.0, 0.0]
limit_rate = mul_limit_rate * max_idx / 100

trading_info, money_list, value_list = getYield(ohlcv_data[data_start+start_diff:], y_pred)

print("거래횟수: %s, %s 회" % (trading_info[1][0], trading_info[1][1]))
print("승률: %s, %s %%" % (trading_info[2][0], trading_info[2][1]))
trading_yield = (money_list[-1]/money_list[0] - 1) * 100
print("수익률: %s %%" % trading_yield)

plt.clf()
plt.plot(list(range(len(value_list))), value_list/value_list[0])
plt.plot(list(range(len(money_list))), money_list/money_list[0])
plt.show()

print("max_idx, max_yield: ", max_idx, max_yield)
print("threshold: ", threshold)
print("limit_rate: ", limit_rate * 100)

plt.clf()
plt.plot(list(range(len(yields[1:]))), yields[1:])
plt.show()

plt.clf()
plt.plot(list(range(len(win_rates[1:]))), win_rates[1:])
plt.show()

In [ ]:
# Threshold, Limit Rate 함께
y_pred = y_preds[:,0]

num_mul = 30

yields = np.zeros((num_mul, num_mul))
win_rates = np.zeros((num_mul, num_mul, 2))

margin_rate = 1
fee_rate = 0.02 / 100
mul_threshold = 5.0 / num_mul
mul_limit_rate = 1.0 / num_mul

for i_th in range(num_mul):
    threshold = [0.0 + mul_threshold * i_th, 0.0 - mul_threshold * i_th]
    for i_li in range(num_mul):
        limit_rate = mul_limit_rate * i_li / 100
        trading_info, money_list, value_list = getYield(ohlcv_data[data_start+start_diff:], y_pred)
        
        yields[i_th, i_li] = (money_list[-1]/money_list[0] - 1) * 100
        win_rates[i_th, i_li] = trading_info[2]

        plt.clf()
        plt.plot(list(range(len(value_list))), value_list/value_list[0])
        plt.plot(list(range(len(money_list))), money_list/money_list[0])
        plt.show()

In [ ]:
max_idx = [0,0]
max_yield = 0
for i_th in range(num_mul):
    current_max = np.max(yields[i_th,1:])
    if max_yield < current_max:
        max_yield = current_max
        max_idx = [i_th, np.argmax(yields[i_th,1:])+1]

print("max_idx, max_yield: ", max_idx, max_yield)

threshold = [0.0 + mul_threshold * max_idx[0], 0.0 - mul_threshold * max_idx[0]]
limit_rate = mul_limit_rate * max_idx[1] / 100

print("threshold: ", threshold)
print("limit_rate: ", limit_rate * 100)
print()

trading_info, money_list, value_list = getYield(ohlcv_data[data_start+start_diff:], y_pred)

print("거래횟수: %s, %s 회" % (trading_info[1][0], trading_info[1][1]))
print("승률: %s, %s %%" % (trading_info[2][0], trading_info[2][1]))
trading_yield = (money_list[-1]/money_list[0] - 1) * 100
print("수익률: %s %%" % trading_yield)

plt.clf()
plt.plot(list(range(len(value_list))), value_list/value_list[0])
plt.plot(list(range(len(money_list))), money_list/money_list[0])
plt.show()

plt.clf()
plt.plot(list(range(len(yields[1:]))), yields[1:])
plt.show()

plt.clf()
plt.plot(list(range(len(win_rates[1:]))), win_rates[1:])
plt.show()

In [ ]:
start = 0
period = 1000

plt.clf()
plt.plot(list(range(period)), y_data[start:start+period])
plt.plot(list(range(period)), y_pred[start:start+period])
plt.show()